<a href="https://colab.research.google.com/github/SLCFLAB/Data-Science-Python/blob/main/Day%204/answers/4_2.finance_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
!pip install finance-datareader
import FinanceDataReader as fdr
import numpy as np
import os
import requests
import re

# References
https://docs.python-requests.org/en/master/

https://gils-lab.tistory.com/

https://github.com/FinanceData/FinanceDataReader

https://github.com/FinanceData/OpenDartReader

# 투자지표 수집
### Finance data reader 활용

In [36]:
# 애플 주식
fdr.DataReader('AAPL', '2017').head(3)

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2017-01-03,29.04,28.95,29.08,28.69,115130000.0,0.0031
2017-01-04,29.00,28.96,29.13,28.94,84470000.0,-0.0014
2017-01-05,29.15,28.98,29.22,28.95,88770000.0,0.0052


In [37]:
# 아마존 주식 - 기간 설정
fdr.DataReader('AMZN', '2017-01-01', '2019-12-31').head(3)

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2017-01-03,753.67,757.92,758.76,747.70,3520000.0,0.0051
2017-01-04,757.18,758.39,759.68,754.20,2510000.0,0.0047
2017-01-05,780.45,761.55,782.40,760.26,5830000.0,0.0307


In [38]:
# 달러 환율 : 95년 1월 1일부터 현재까지로 기간 설정
fdr.DataReader('USD/KRW', '1995-01-01').head(3)

,Close,Open,High,Low,Change
Date,,,,,
1995-01-02,788.50,788.50,788.50,788.50,0.0000
1995-01-03,788.65,788.05,788.85,787.55,0.0002
1995-01-04,789.95,788.65,790.05,788.45,0.0016


In [39]:
stocks = fdr.StockListing('KRX') # 코스피, 코스닥, 코넥스 전체

In [40]:
# fdr 을 통해 받아온 데이터 확인
print(len(stocks)) # 주식의 수

stocks.head(5)

7912


,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,"박대현, 손삼달",http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도


In [41]:
stocks.dropna(inplace = True) # 지표가 없는 선물 등은 홈페이지, 지역, 산업 등이 모두 결측으로 되어 있으므로 삭제하면 됨

In [42]:
# 아이투자 사이트에서 주식에 관한 추가적인 정보 불러오기
# 주식의 symbol은 fdr에서 읽어온 정보를 활용

# base_url 지정
# {} 부분에 주식의 symbol를 넣어서 이투자에서 크롤링할 준비 (e.g. 060310)
# https://search.itooza.com/search.htm?seName=060310&jl=&search_ck=&sm=&sd=2021-08-09&ed=2021-09-08&ds_de=&page=&cpv=

base_url = "https://search.itooza.com/search.htm?seName={}&jl=&search_ck=&sm=&sd=2021-08-09&ed=2021-09-08&ds_de=&page=&cpv="

formatted_url = base_url.format('060310') # S3 의 주식 symbol 
html = requests.get(formatted_url, 
                    headers = {'User-Agent':'Gils'} # 자신의 User-Agent로 넣어주는 것이 좋음. 안 넣어줘도 크롤링되면 상관없음
                    ).content
soup = BeautifulSoup(html, 'lxml')


In [ ]:
for code, name in stocks[:5][['Symbol', 'Name']].values:
    print(code, name)

    # base_url에 code를 받아서 requests로 요청
    # 받은 html 코드를 BeautifulSoup 객체로 만듬
    while True:
        try:
            url = base_url.format(code)
            print(code, url)
            html = requests.get(url, headers={'User-Agent':'Gils'}).content
            soup = BeautifulSoup(html, 'lxml')
            break
        except:
            print("Request error occurred. 15 seconds break")
            time.sleep(15)
    
    # 만약 해당 종목 자료가 없다면, 아래 테이블이 없음
    table = soup.find("table", {"class":"ex"}) # class가 ex인 테이블을 찾음

    if table == None: # class가 ex인 테이블이 없으면 크롤링하지 않음
        continue

    idx = ["2021-2Q", "2021-1Q",
           "2020-4Q", "2020-3Q", "2020-2Q", "2020-1Q",
           "2019-4Q", "2019-3Q", "2019-2Q", "2019-1Q",
           "2018-4Q", "2018-3Q"] # 인덱스를 직접 정의

    table_body = table.find('tbody')  # table에서 tbody만 가져옴
    tr_list = table_body.find_all('tr') # tbody에서 행을 다 찾아서 저장. tr_list 는 태그들을 모아놓은 리스트 형태.

    result_dict = {}
    for tr in tr_list:
        col_name = tr.find('th').text # 투자 지표 이름
        values = [td.text for td in tr.find_all('td')] # tr에 있는 모든 td의 text만 가져와서 저장
        result_dict[col_name] = values # 사전으로 저장

    result = pd.DataFrame.from_dict(result_dict)
    result.drop('주가', axis = 1, inplace = True) # 주가열도 필요없으니 삭제
    result['분기'] = idx
    
    if code == '060310': print(result) # 첫 루프를 돌 때만 result 출력

    result.to_csv(name + ".csv", index = False) # csv 파일로 저장. 파일 목록 확인해보기
    
    time.sleep(1) # 1초씩 재우기

# 한경컨센서스 리포트 수집

In [44]:
stocks = fdr.StockListing('KRX') # 코스피, 코스닥, 코넥스 전체

In [45]:
stocks['Symbol'] = stocks['Symbol'].astype(str)

# 리포트 제목에서 코드 번호와 이름 따로 저장.
def remove_noise_and_split_title(title):
    in_code = ''
    in_name = ''
    
    for code, name in stocks[['Symbol', 'Name']].values:
        if code in title and name in title:
            in_code = code
            in_name = name
    
    # 한글, 영어, 숫자 외 노이즈 제거
    clean_title = re.sub('[^A-Za-z0-9가-힣]', ' ', title)
    
    # 기업명 코드 수정
    clean_title = clean_title.replace(in_code, ' ')
    clean_title = clean_title.replace(in_name, ' ')
    while ' ' * 2 in clean_title:
        clean_title = clean_title.replace(' ' * 2, ' ')
    
    if in_name == '': # 기업명이 없는 제목이라면, 데이터에 추가하지 않음
        return [None] 
    else:
        return [in_name, in_code, clean_title]  

In [46]:
remove_noise_and_split_title("이마트(139480)플랫폼으로 진화하는 SSG.Com")

['이마트', '139480', ' 플랫폼으로 진화하는 SSG Com']

In [47]:
remove_noise_and_split_title("시디즈(134790)안정적인 성장성 & 저평가")

['시디즈', '134790', ' 안정적인 성장성 저평가']

In [48]:
base_url = "http://consensus.hankyung.com/apps.analysis/analysis.list?&sdate=2016-09-01&edate=2021-08-31&report_type=CO&order_type=&now_page={}"
data = [] 

for page_no in range(1, 3):    #1,2 페이지만 크롤링 
    '''
    To-do
    url = # base_url에서 page_no를 넣어서 url이라는 변수에 저장
    html = # requests로 url 을 넣어 content 받아오기
    soup = # Beautifulsoup 객체로 저장. 파서는 lxml 사용
    '''
    
    # 예시 정답 
    url = base_url.format(page_no) # base_url에서 page_no를 넣어서 url이라는 변수에 저장
    html = requests.get(url, headers={'User-Agent':'Gils'}).content # requests로 url 을 넣어 content 받아오기
    soup = BeautifulSoup(html, 'lxml') # Beautifulsoup 객체로 저장. 파서는 lxml 사용
    
    print("{}/{}".format(page_no, 4284))


    '''
    To-do
    table = # div 의 클래스 이름이 'table_style_01'인 태그를 고르고 그 태그 안에서 table을 찾기
            # hint : soup.select() 사용 OR soup.find().find() 사용 
    '''

    # 예시 정답
    table = soup.find("div", {"class":"table_style01"}).find('table')
    #table = soup.select_one('div.table_style01 > table')

    for tr in table.find_all("tr")[1:]: # 1번째 행부터 순회
        record = []
        for i, td in enumerate(tr.find_all("td")[:6]): # 6번째 셀까지 순회
            if i == 1:
                record += remove_noise_and_split_title(td.text) # remove_noise_title의 출력과 이어 붙임
            elif i == 3: # 노이즈가 껴있는 세번째 셀만 따로 처리
                record.append(td.text.replace(" ", "").replace("\r","").replace("\n",""))
            else: # 1번째 셀이 아니면:
                record.append(td.text) # 셀의 텍스트 값 그대로 입력
            
        if None not in record: # 레코드에 None이 없으면
            data.append(record)

    time.sleep(1) # 연결 끊김 방지를 위해, 1초씩 재움
    
data = pd.DataFrame(data, columns = ["작성일", "종목명", "종목코드", "제목", "적정가격", "평가의견", "작성자", "작성기관"])
data.to_csv("리포트_데이터.csv", index = False)

1/4284
2/4284


select 문법 참고
https://greenfishblog.tistory.com/193#google_vignette 

# OpenDart 활용

In [ ]:
!pip install opendartreader
import OpenDartReader

API 키 발급 링크 : https://opendart.fss.or.kr/uss/umt/EgovMberInsertView.do

In [50]:
# 위 링크에서 발급받은 api 키 입력
my_api = "1b61ac20a39bacd96911f4c99d32245927ea0d8a"
# my_api = "7c21274166c816929c5a5c6dacffaa8315b7daeb"
dart = OpenDartReader(my_api)

In [ ]:
# 돌리는데 시간이 오래걸림. 

# == 1. 공시정보 검색 ==
# 삼성전자 2019-07-01 하루 동안 공시 목록 (날짜에 다양한 포맷이 가능합니다)
dart.list('005930', end='2016-7-1')

# 삼성전자 상장이후 모든 공시 목록 (5,142 건+)
dart.list('005930', start='1900') 

# 삼성전자 2019-01-01 ~ 2019-12-31 모든 공시 목록
dart.list('005930', start='2019-01-01', end='2019-12-31') 

# 삼성전자 1999-01-01 이후 모든 정기보고서
dart.list('005930', start='1999-01-01', kind='A', final=False)

# 삼성전자 1999년~2019년 모든 정기보고서(최종보고서)
dart.list('005930', start='1999-01-01', end='2019-12-31', kind='A') 

# 회사명에 삼성전자가 포함된 회사들에 대한 개황정보
dart.company_by_name('삼성전자')


# ==== 2. 사업보고서 ====
# 삼성전자(005930), 배당관련 사항, 2018년
dart.report('005930', '배당', 2018) 

# 서울반도체(046890), 최대주주 관한 사항, 2018년
dart.report('046890', '최대주주', 2018) 


In [65]:
# 기업, 배당에 관한 정보, 연도
dart.report("SK하이닉스","배당",2020).head(3)

# thstrm, frmtrm, lwfr 은 각각 전전기, 전기, 당기


,rcept_no,corp_cls,corp_code,corp_name,se,thstrm,frmtrm,lwfr
0,20210330000776,Y,00164779,SK하이닉스,주당액면가액(원),"5,000","5,000","5,000"
1,20210330000776,Y,00164779,SK하이닉스,(연결)당기순이익(백만원),"4,755,102","2,005,975","15,540,111"
2,20210330000776,Y,00164779,SK하이닉스,(별도)당기순이익(백만원),"4,217,841","1,476,981","15,407,086"


In [52]:
'''
To-do
fdr로 한국 내의 코스피, 코스닥, 코넥스 리스트 받아와 stock_list 에 저장
stock_list = 
'''

# 예시 정답
stock_list = fdr.StockListing("KRX")

In [53]:
'''
To-do
stock_list 데이터프레임에서 Market이 코스피나 코스닥인 것들만 추려내기 (Hint : loc 함수와 isin 함수 사용)
stock_list = 
'''

# 예시 정답
stock_list = stock_list.loc[stock_list['Market'].isin(["KOSPI","KOSDAQ"])]


In [59]:
# Region이 null이 아닌 데이터에서 Name 칼럼만 뽑아내어 stock_name_list에 저장
stock_name_list = stock_list.loc[stock_list['Region'].notnull(),'Name'].tolist()
print(stock_name_list[:5])

['3S', 'AJ네트웍스', 'AK홀딩스', 'APS홀딩스', 'AP시스템']


In [55]:
# 주식 이름을 받아 report에서 찾아 당기, 전기, 전전기 배당을 리턴해주는 함수
def find_dividends(stock_name, year):
    stock_name_report = dart.report(stock_name, "배당", year, "11011")
    if stock_name_report is None:
        return np.nan, np.nan, np.nan
    else:
        ''' 
        To-do
        stock_name_report에서 se 칼럼이 "주당 현금배당금(원)" 인 행들을 선택하면 두 개의 행이 나오는데 그중 첫번째 행 선택
        stock_name_report = stock_name_report.loc[~~].iloc[0]
        '''
        # 예시 정답
        stock_name_report = stock_name_report.loc[stock_name_report['se']=="주당 현금배당금(원)"].iloc[0]

        ''' 
        To-do
        stock_name_report의 각 thstrm, frmtrm, lwfr 열들을 보면 쉼표(,) 와 공백(-)가 있음. 쉼표는 없애고 공백은 0으로 바꾼 후, 정수로 형변환
        thstrm_dividends = 
        frmtrm_dividends = 
        lwfr_dividends = 
        '''
    
    # thstrm : 당기, frmtrm : 전기, lwfr : 전전기
    # 예시 정답
    thstrm_dividends = int(stock_name_report['thstrm'].replace('-','0').replace(',',''))
    frmtrm_dividends = int(stock_name_report['frmtrm'].replace('-','0').replace(',',''))
    lwfr_dividends = int(stock_name_report['lwfr'].replace('-','0').replace(',',''))
    
    return lwfr_dividends, frmtrm_dividends, thstrm_dividends

In [56]:
find_dividends("삼성전자",2020)

(1416, 1416, 2994)

In [ ]:
data = []
for idx, stock_name in enumerate(stock_name_list[0:5]):
    print(idx+1, stock_name)
    record = [stock_name]

    for year in [2015,2018,2020]:
    # 위에서 만든 find_dividends 함수 사용하여 배당금 받아옴
        lwfr_dividends, frmtrm_dividends, thstrm_dividends = find_dividends(stock_name, year)

        '''
        To-do
        2013년부터 2020년까지 배당금 모두 받아와서 record 리스트에 추가
        Hint : 2015, 2018년의 경우 전전기 배당금, 전기 배당금, 당기 배당금 받아야 하고 2020년의 경우 전기와 당기 배당금을 받아와 record에 추가
        '''
        
        if year != 2020:
              record += [lwfr_dividends, frmtrm_dividends, thstrm_dividends]
        else:
              record += [frmtrm_dividends, thstrm_dividends]

    data.append(record)

    time.sleep(0.5)
    print(record)
  
  # 몇몇 기업은 배당금이 nan 또는 0으로 나오는 경우 있음.

In [58]:
# 칼럼명 지정
data = pd.DataFrame(data, columns=["stock_name","2015","2016","2017","2018","2019","2020","2021","2022"])

# 배당목록.csv로 저장
data.to_csv("배당목록.csv")

# 주요 재무제표 수집 및 가공

In [ ]:
# finstate : 상장기업의 재무제표 불러옴
dart.finstate("LG전자", 2020,"11011").head(5)

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20210316000661,11011,2020,00401731,066570,CFS,연결재무제표,BS,재무상태표,유동자산,제 19 기,2020.12.31 현재,"23,239,420,000,000",제 18 기,2019.12.31 현재,"19,753,485,000,000",제 17 기,2018.12.31 현재,"19,362,854,000,000",1
1,20210316000661,11011,2020,00401731,066570,CFS,연결재무제표,BS,재무상태표,비유동자산,제 19 기,2020.12.31 현재,"24,964,807,000,000",제 18 기,2019.12.31 현재,"25,106,390,000,000",제 17 기,2018.12.31 현재,"24,965,589,000,000",3
2,20210316000661,11011,2020,00401731,066570,CFS,연결재무제표,BS,재무상태표,자산총계,제 19 기,2020.12.31 현재,"48,204,227,000,000",제 18 기,2019.12.31 현재,"44,859,875,000,000",제 17 기,2018.12.31 현재,"44,328,443,000,000",5
3,20210316000661,11011,2020,00401731,066570,CFS,연결재무제표,BS,재무상태표,유동부채,제 19 기,2020.12.31 현재,"20,207,492,000,000",제 18 기,2019.12.31 현재,"17,657,916,000,000",제 17 기,2018.12.31 현재,"17,135,029,000,000",7
4,20210316000661,11011,2020,00401731,066570,CFS,연결재무제표,BS,재무상태표,비유동부채,제 19 기,2020.12.31 현재,"10,454,610,000,000",제 18 기,2019.12.31 현재,"10,776,821,000,000",제 17 기,2018.12.31 현재,"10,886,507,000,000",9


In [ ]:
result = dart.finstate("LG전자", 2020,"11011")

'''
To-do
1. result 에서 fs_nm 칼럼이 연결재무제표인 행들만 저장
2. result 에서 account_nm 칼럼이 당기순이익 or 영업이익 or 매출액인 행들만 저장
result = ~
result = ~
'''

# 예시 정답
result = result.loc[result['fs_nm'] == "연결재무제표"]
result = result.loc[result['account_nm'].isin(['당기순이익','영업이익','매출액'])]

# result에서 아래 칼럼들만 뽑아옴
display("before", result)
result = result.loc[:,["account_nm","fs_nm","sj_nm","thstrm_dt","thstrm_amount"]]

display("after", result)


'before'

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
9,20210316000661,11011,2020,00401731,066570,CFS,연결재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,"63,262,046,000,000",제 18 기,2019.01.01 ~ 2019.12.31,"62,306,175,000,000",제 17 기,2018.01.01 ~ 2018.12.31,"61,341,664,000,000",23
10,20210316000661,11011,2020,00401731,066570,CFS,연결재무제표,IS,손익계산서,영업이익,제 19 기,2020.01.01 ~ 2020.12.31,"3,194,987,000,000",제 18 기,2019.01.01 ~ 2019.12.31,"2,436,139,000,000",제 17 기,2018.01.01 ~ 2018.12.31,"2,703,291,000,000",25
12,20210316000661,11011,2020,00401731,066570,CFS,연결재무제표,IS,손익계산서,당기순이익,제 19 기,2020.01.01 ~ 2020.12.31,"2,063,790,000,000",제 18 기,2019.01.01 ~ 2019.12.31,"179,948,000,000",제 17 기,2018.01.01 ~ 2018.12.31,"1,472,814,000,000",29


'after'

,account_nm,fs_nm,sj_nm,thstrm_dt,thstrm_amount
9,매출액,연결재무제표,손익계산서,2020.01.01 ~ 2020.12.31,"63,262,046,000,000"
10,영업이익,연결재무제표,손익계산서,2020.01.01 ~ 2020.12.31,"3,194,987,000,000"
12,당기순이익,연결재무제표,손익계산서,2020.01.01 ~ 2020.12.31,"2,063,790,000,000"


In [ ]:
stock_list = stock_list.loc[stock_list['Market'].isin(["KOSPI","KOSDAQ"])]
stock_name_list = stock_list.loc[stock_list['Region'].notnull(),'Name'].tolist()
stock_name_list[:5]

['3S', 'AJ네트웍스', 'AK홀딩스', 'APS홀딩스', 'AP시스템']

In [ ]:
def find_financial_indicators(stock_name, year, indicators):
    report = dart.finstate(stock_name, year)
    if report is None:
        data = [[stock_name, year] + [np.nan] * len(indicators)]
        data = [[stock_name, year-1] + [np.nan] * len(indicators)]
        data = [[stock_name, year-2] + [np.nan] * len(indicators)]
        
        return pd.DataFrame(data, columns = ["기업", "연도"] + indicators)
    
    else:
        # 원하는 indicators 만 뽑아옴 (["자산총계", "부채총계", "자본총계", "매출액", "영업이익", "당기순이익"])
        report = report[report['account_nm'].isin(indicators)]
        # print(report)

        # 연결재무제표가 있다면 연결재무제표 사용, 없다면 재무제표 사용
        if sum(report['fs_nm'] == "연결재무제표") >0:
            report = report.loc[report['fs_nm'] == "연결재무제표"]
        
        else:
            report = report.loc[report['fs_nm'] == "재무제표"]
        
        data = []
        for y,c in zip([year, year-1, year-2], ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']):
            record = [stock_name, y]
            for indic in indicators:
                if sum(report['account_nm'] == indic) > 0:
                    value = report.loc[report['account_nm'] == indic, c].iloc[0]
                else:
                    value = np.nan
                record.append(value)
                
            data.append(record)
        return pd.DataFrame(data,columns = ["기업", "연도"] + indicators)

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
0,삼성전자,2020,"378,235,718,000,000","102,287,702,000,000","275,948,016,000,000","236,806,988,000,000","35,993,876,000,000","26,407,832,000,000"
1,삼성전자,2019,"352,564,497,000,000","89,684,076,000,000","262,880,421,000,000","230,400,881,000,000","27,768,509,000,000","21,738,865,000,000"
2,삼성전자,2018,"339,357,244,000,000","91,604,067,000,000","247,753,177,000,000","243,771,415,000,000","58,886,669,000,000","44,344,857,000,000"


In [ ]:
indicators = ["자산총계", "부채총계", "자본총계", "매출액", "영업이익", "당기순이익"]
display(find_financial_indicators("삼성전자", 2020, indicators))

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
0,삼성전자,2020,"378,235,718,000,000","102,287,702,000,000","275,948,016,000,000","236,806,988,000,000","35,993,876,000,000","26,407,832,000,000"
1,삼성전자,2019,"352,564,497,000,000","89,684,076,000,000","262,880,421,000,000","230,400,881,000,000","27,768,509,000,000","21,738,865,000,000"
2,삼성전자,2018,"339,357,244,000,000","91,604,067,000,000","247,753,177,000,000","243,771,415,000,000","58,886,669,000,000","44,344,857,000,000"


In [ ]:
data = pd.DataFrame()

for idx, stock_name in enumerate(stock_name_list[:5]):
    print(idx+1, stock_name)
    for year in [2015,2018,2020]:
        try:
            result = find_financial_indicators(stock_name, year, indicators)
        except:
            pass
        data = pd.concat([data, result], axis=0, ignore_index=True)
        time.sleep(0.5)

1 3S
2 AJ네트웍스
3 AK홀딩스
4 APS홀딩스
5 AP시스템


In [ ]:
# 중복 행 제거
data.drop_duplicates(inplace = True)

In [ ]:
def str_to_float(value):
    if type(value) == float:
        return value
    elif value == "-":
        return 0
    else:
        return float(value.replace(',',''))

for indc in indicators:
    data[indc] = data[indc].apply(str_to_float)

In [ ]:
print(data.head(5))

   기업    연도          자산총계          부채총계          자본총계           매출액  \
0  3S  2015  6.735997e+10  2.942016e+10  3.793981e+10  2.321054e+10   
1  3S  2014  7.131201e+10  2.831919e+10  4.299283e+10  2.886299e+10   
2  3S  2013  6.766058e+10  1.988388e+10  4.777671e+10  3.223645e+10   
3  3S  2018  4.952100e+10  1.876866e+10  3.075234e+10  3.154617e+10   
4  3S  2017  5.370690e+10  2.452724e+10  2.917967e+10  2.648629e+10   

           영업이익         당기순이익  
0 -4.157989e+09 -5.599033e+09  
1 -2.540740e+09 -4.760633e+09  
2  1.859291e+09  7.011469e+08  
3  3.333106e+08 -1.432240e+09  
4 -2.421225e+09 -8.803810e+09  


In [ ]:
# 부채 비율 계산
data['부채비율'] = data['자본총계'] / data['자산총계'] * 100
display(data['부채비율'].head())

0    56.323967
1    60.288336
2    70.612320
3    62.099593
4    54.331315
Name: 부채비율, dtype: float64

In [ ]:
# 영업이익 증가율
data.sort_values(by=['기업','연도'], inplace=True)

''' 
To-do
영업이익증가율 = 전년도 영업이익과의 차이 / 영업이익
data에 새로운 칼럼을 만듬
data['영업이익증가율'] = ~
'''

# 예시 정답
data['영업이익증가율'] = data['영업이익'].diff() / data['영업이익']

# data 연도가 2013년인 행들은 np.nan 으로 둠. (증가율을 비교할 2012년 데이터가 없기 때문)
data.loc[data['연도'] == 2013, '영업이익증가율'] = np.nan

In [ ]:
# 매출액
data.sort_values(by=['기업','연도'], inplace=True)

''' 
To-do
매출액증가율 = 전년도 매출액과의 차이 / 매출액
data에 새로운 칼럼을 만듬
data['매출액증가율'] = ~
'''

# 예시 정답
data['매출액증가율'] = data['매출액'].diff() / data['매출액']

data.loc[data['연도'] == 2013, '매출액증가율'] = np.nan

In [ ]:
# 당기순이익
data.sort_values(by=['기업','연도'], inplace=True)
data['당기순이익증가율'] = data['당기순이익'].diff() / data['당기순이익']
data.loc[data['연도'] == 2013, '당기순이익증가율'] = np.nan

In [ ]:
data.head(5)

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,부채비율,영업이익증가율,매출액증가율,당기순이익증가율,매출액_상태
2,3S,2013,6.766058e+10,1.988388e+10,4.777671e+10,3.223645e+10,1.859291e+09,7.011469e+08,70.612320,NaN,NaN,NaN,NaN
1,3S,2014,7.131201e+10,2.831919e+10,4.299283e+10,2.886299e+10,-2.540740e+09,-4.760633e+09,60.288336,1.731791,-0.116878,1.147280,흑자지속
0,3S,2015,6.735997e+10,2.942016e+10,3.793981e+10,2.321054e+10,-4.157989e+09,-5.599033e+09,56.323967,0.388950,-0.243529,0.149740,흑자지속
5,3S,2016,5.774639e+10,2.314263e+10,3.460376e+10,2.374242e+10,1.176536e+09,-3.403438e+09,59.923676,4.534095,0.022402,-0.645111,흑자지속
4,3S,2017,5.370690e+10,2.452724e+10,2.917967e+10,2.648629e+10,-2.421225e+09,-8.803810e+09,54.331315,1.485926,0.103596,0.613413,흑자지속


In [ ]:
# ROA

data ["ROA"] = data['당기순이익'] / data['자산총계']

# ROE
average_equity = data['자본총계'].rolling(2).mean()
data['ROE'] = data['당기순이익'] / average_equity


In [ ]:
data.to_csv("주요재무지표.csv", index=False)

# 전체 재무제표 수집

In [ ]:
stock_list = fdr.StockListing("KRX").dropna()

In [ ]:
# 숫자로 모두 변환
def convert_str_to_float(value):
    if type(value) == float: # nan의 자료형은 float임
        return value
    elif value == '-': # -로 되어 있으면 0으로 변환
        return 0
    else:
        return float(value.replace(',', ''))

In [ ]:
def extract_info_and_save(dart, path, year, name, code, report_type, report_type_code):
    report = dart.finstate(code, year, report_type_code)
    if report is None:
        pass
    else:
        submission_date = report['rcept_no'].iloc[0][:8]
        report = report[['fs_nm', 'account_nm', 'thstrm_dt', 'thstrm_amount', 'sj_nm']]
        report.rename({"fs_nm":"개별/연결",
                       "account_nm":"계정명",
                       "sj_nm":"재무제표명",
                       "thstrm_dt": "당기일자",
                       "thstrm_amount":"금액"}, axis = 1, inplace = True)
        report['금액'] = report['금액'].apply(convert_str_to_float)
        
        if name not in os.listdir(path):
            os.mkdir(path + "/" + name)
            
        report.to_csv(path + "/{}/{}_{}년_{}.csv".format(name, submission_date, year, report_type),
                      index = False)

In [ ]:
dart = OpenDartReader(my_api)
for code, name in stock_list[:2][['Symbol', 'Name']].values:
    for year in range(2016, 2022):
        for report_type, report_type_code in zip(["1분기보고서", "반기보고서", "3분기보고서", "사업보고서"],
                                                  ["11013", "11012", "11014", "11011"]):
            print(name, year, report_type)
            while True:
                try:
                    extract_info_and_save(dart = dart,
                                          path = r".",
                                          year = year,
                                          name = name,
                                          code = code,
                                          report_type = report_type,
                                          report_type_code = report_type_code)
                    time.sleep(0.5)
                    break
                    
                except:
                    time.sleep(60)